In [1]:
import pandas as pd
import plotly.express as px
import pycountry
import plotly.express as px
import plotly.graph_objs as go
import warnings
from pycountry_convert import country_alpha2_to_continent_code, convert_continent_code_to_continent_name
warnings.filterwarnings('ignore')

In [2]:
demographics = pd.read_csv('../data/demographics.csv')
demographics = demographics[demographics['Country of origin (ISO)'] != demographics['Country of asylum (ISO)']]

In [3]:
# change the column names from ISO to country
def iso_to_continent(country_iso):
    try:
        country = pycountry.countries.get(alpha_3=country_iso)
        country_alpha2 = country.alpha_2
        continent_code = country_alpha2_to_continent_code(country_alpha2)
        continent_name = convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except (KeyError, AttributeError):
        return None


#create a new column 'continent' by applying the conversion function to the 'Country of asylum (ISO)' column
demographics['continent'] = demographics['Country of asylum (ISO)'].apply(iso_to_continent)

#demographics

In [4]:
#aggregate the data by 'Year', 'Country of asylum (ISO)', and 'continent', summing the values for each age group and gender
grouped_data = demographics.groupby(['Year', 'Country of asylum (ISO)', 'continent']).agg({
    'Female 0 - 4': 'sum',
    'Female 5 - 11': 'sum',
    'Female 12 - 17': 'sum',
    'Female 18 - 59': 'sum',
    'Female 60': 'sum',
    'Female total': 'sum',
    'Male 0 - 4': 'sum',
    'Male 5 - 11': 'sum',
    'Male 12 - 17': 'sum',
    'Male 18 - 59': 'sum',
    'Male 60': 'sum',
    'Male total': 'sum',
    'Total': 'sum'
}).reset_index()

#add a new column with country names
grouped_data['Country of asylum'] = grouped_data['Country of asylum (ISO)'].apply(lambda x: pycountry.countries.get(alpha_3=x).name)

# Create a bubble chart using Plotly Express with a slider for the 'Year' column
fig = px.scatter(grouped_data, x='Female total', y='Male total', size='Total', 
                 color='continent', hover_name='Country of asylum', log_x=True, log_y=True, 
                 size_max=60, title='Total Population by Gender in Each Country, Colored by Continent',
                 animation_frame='Year', range_x=[grouped_data['Female total'].min()+1, grouped_data['Female total'].max()],
                 range_y=[grouped_data['Male total'].min()+1, grouped_data['Male total'].max()])

#add a line y=x
min_value = min(grouped_data['Female total'].min(), grouped_data['Male total'].min())
max_value = max(grouped_data['Female total'].max(), grouped_data['Male total'].max())

fig.add_trace(go.Scatter(x=[min_value, max_value],
                         y=[min_value, max_value],
                         mode='lines',
                         name='y=x',
                         line=dict(dash='dash',color = 'black', width=1),
                         showlegend=False),
                         )
#show the bubble chart
fig.update_layout(
    template="plotly_white",
    margin=dict(l=20, r=100, t=80, b=20),
    xaxis=dict(title='Female Total', autorange=True),
    yaxis=dict(title='Male Total', autorange=True),
    width=1200, height=700,
    title=dict(
        font=dict(
            family="Arial",
            size=20
        )
    ),
    xaxis_title=dict(
        font=dict(
            family="Arial",
            size=18
        )
    ),
    yaxis_title=dict(
        font=dict(
            family="Arial",
            size=18
        )
    ),
)

fig.show()
